In [42]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Border, Side

file_2022 = "2022_年間売上表.xlsx"
file_2023 = "2023_年間売上表.xlsx"
output_xlsx = "売上集計表.xlsx"
header_color = "F2F2F2" # 薄いグレー

try:
    df_2022 = pd.read_excel(file_2022)
    df_2023 = pd.read_excel(file_2023)

    combined_df = pd.concat([df_2022, df_2023], ignore_index=True)

    summary_df = combined_df.groupby(['商品', '売上年'])['金額（千円）'].sum().reset_index()

    summary_df.rename(columns={'金額（千円）': '合計金額（千円）'}, inplace=True)

except FileNotFoundError:
    exit()

wb = Workbook()
ws = wb.active
ws.title = "年間売上合計"

header_fill = PatternFill(start_color=header_color, end_color=header_color, fill_type="solid")
header_font = Font(bold=True)

thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                     top=Side(style='thin'), bottom=Side(style='thin'))

header = summary_df.columns.tolist()
ws.append(header)

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.border = thin_border

for row_index, row_data in summary_df.iterrows():
    ws.append(row_data.tolist())

    for cell in ws[row_index + 2]:
        cell.border = thin_border

        if cell.column_letter == 'C':
            cell.number_format = '#,##0'

for col in ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length * 1.2)
    ws.column_dimensions[column].width = adjusted_width

try:
    wb.save(output_xlsx)
except Exception as e:
    print(f"❌ ファイルの保存中にエラーが発生しました: {e}")